In [1]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib
import seaborn

In [2]:
import os
print(os.getcwd())

/Users/yaohuazhao/SimVis


In [3]:
#windows
#building_metadata_df = pd.read_csv("/Users/yaohuazhao/Downloads/ashrae-energy-prediction/building_metadata.csv")
#test_df = pd.read_csv("/Users/yaohuazhao/Downloads/ashrae-energy-prediction/test.csv")
#train_df = pd.read_csv("/Users/yaohuazhao/Downloads/ashrae-energy-prediction/train.csv")
#weather_train_df = pd.read_csv("/Users/yaohuazhao/Downloads/ashrae-energy-prediction/weather_train.csv")
#weather_test_df = pd.read_csv("/Users/yaohuazhao/Downloads/ashrae-energy-prediction/weather_test.csv")

In [4]:
#Mac
building_metadata_df = pd.read_csv("/Users/yaohuazhao/Downloads/ashrae-energy-prediction/building_metadata.csv")
test_df = pd.read_csv("/Users/yaohuazhao/Downloads/ashrae-energy-prediction/test.csv")
train_df = pd.read_csv("/Users/yaohuazhao/Downloads/ashrae-energy-prediction/train.csv")
weather_train_df = pd.read_csv("/Users/yaohuazhao/Downloads/ashrae-energy-prediction/weather_train.csv")
weather_test_df = pd.read_csv("/Users/yaohuazhao/Downloads/ashrae-energy-prediction/weather_test.csv")


In [5]:
#Data dimensions
print("buiding_metadata: ",building_metadata_df.shape)
print("test: ",test_df.shape)
print("train: ",train_df.shape)
print("weather_train: ",weather_train_df.shape)
print("weather_test: ",weather_test_df.shape)

buiding_metadata:  (1449, 6)
test:  (41697600, 4)
train:  (20216100, 4)
weather_train:  (139773, 9)
weather_test:  (277243, 9)


In [6]:
#sample data
building_metadata_df.head(10)


,site_id,building_id,primary_use,square_feet,year_built,floor_count
0,0,0,Education,7432,2008.0,NaN
1,0,1,Education,2720,2004.0,NaN
2,0,2,Education,5376,1991.0,NaN
3,0,3,Education,23685,2002.0,NaN
4,0,4,Education,116607,1975.0,NaN
5,0,5,Education,8000,2000.0,NaN
6,0,6,Lodging/residential,27926,1981.0,NaN
7,0,7,Education,121074,1989.0,NaN
8,0,8,Education,60809,2003.0,NaN
9,0,9,Office,27000,2010.0,NaN


In [7]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41697600 entries, 0 to 41697599
Data columns (total 4 columns):
row_id         int64
building_id    int64
meter          int64
timestamp      object
dtypes: int64(3), object(1)
memory usage: 1.2+ GB


In [8]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20216100 entries, 0 to 20216099
Data columns (total 4 columns):
building_id      int64
meter            int64
timestamp        object
meter_reading    float64
dtypes: float64(1), int64(2), object(1)
memory usage: 616.9+ MB


In [9]:
weather_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 277243 entries, 0 to 277242
Data columns (total 9 columns):
site_id               277243 non-null int64
timestamp             277243 non-null object
air_temperature       277139 non-null float64
cloud_coverage        136795 non-null float64
dew_temperature       276916 non-null float64
precip_depth_1_hr     181655 non-null float64
sea_level_pressure    255978 non-null float64
wind_direction        264873 non-null float64
wind_speed            276783 non-null float64
dtypes: float64(7), int64(1), object(1)
memory usage: 19.0+ MB


In [10]:
weather_train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139773 entries, 0 to 139772
Data columns (total 9 columns):
site_id               139773 non-null int64
timestamp             139773 non-null object
air_temperature       139718 non-null float64
cloud_coverage        70600 non-null float64
dew_temperature       139660 non-null float64
precip_depth_1_hr     89484 non-null float64
sea_level_pressure    129155 non-null float64
wind_direction        133505 non-null float64
wind_speed            139469 non-null float64
dtypes: float64(7), int64(1), object(1)
memory usage: 9.6+ MB


In [11]:
# Reduce memory size
## Function to reduce the DF size
#def reduce_mem_usage(df, verbose=True):
#    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
#    start_mem = df.memory_usage().sum() / 1024**2    
#    for col in df.columns:
#        col_type = df[col].dtypes
#        if col_type in numerics:
#            c_min = df[col].min()
#            c_max = df[col].max()
#            if str(col_type)[:3] == 'int':
#                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
#                    df[col] = df[col].astype(np.int8)
#                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
#                    df[col] = df[col].astype(np.int16)
#                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
#                    df[col] = df[col].astype(np.int32)
#                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
#                    df[col] = df[col].astype(np.int64)  
#            else:
#                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
#                    df[col] = df[col].astype(np.float16)
#                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
#                    df[col] = df[col].astype(np.float32)
#                else:
#                    df[col] = df[col].astype(np.float64)    
#    end_mem = df.memory_usage().sum() / 1024**2
#    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * #(start_mem - end_mem) / start_mem))
#    return df

In [12]:
#train_df = reduce_mem_usage(train_df)
#test_df = reduce_mem_usage(test_df)

#weather_train_df = reduce_mem_usage(weather_train_df)
#weather_test_df = reduce_mem_usage(weather_test_df)
#building_metadata_df = reduce_mem_usage(building_metadata_df)

In [13]:
# join train_df, buiding_metadata_df, weather_train_df
df1 = train_df.merge(building_metadata_df,on="building_id",how="left")
#combined_train = combined_train1.merge(weather_train_df,on="site_id",how="left")
df1.info()
#combined_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20216100 entries, 0 to 20216099
Data columns (total 9 columns):
building_id      int64
meter            int64
timestamp        object
meter_reading    float64
site_id          int64
primary_use      object
square_feet      int64
year_built       float64
floor_count      float64
dtypes: float64(3), int64(4), object(2)
memory usage: 1.5+ GB


In [14]:
df1.isnull().sum()


building_id             0
meter                   0
timestamp               0
meter_reading           0
site_id                 0
primary_use             0
square_feet             0
year_built       12127645
floor_count      16709167
dtype: int64

In [15]:
#sample time
df1.timestamp[0]

'2016-01-01 00:00:00'

In [16]:
#create month and hour columns
df1["month"] = df1["timestamp"].apply(lambda x: int(x[5:7]))
df1["hour"] = df1["timestamp"].apply(lambda x: int(x[11:13]))

In [17]:
#change timestamp string to timestamp object
from datetime import datetime
df1["timestamp_new"] = df1["timestamp"].apply(lambda x: datetime.strptime(x,"%Y-%m-%d %H:%M:%S"))

In [18]:
#add day of week
df1["day_of_week"] = df1["timestamp_new"].apply(lambda x: x.weekday())

In [19]:
#drop unused columns
df1.drop(['timestamp_new'],axis=1,inplace=True)
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20216100 entries, 0 to 20216099
Data columns (total 12 columns):
building_id      int64
meter            int64
timestamp        object
meter_reading    float64
site_id          int64
primary_use      object
square_feet      int64
year_built       float64
floor_count      float64
month            int64
hour             int64
day_of_week      int64
dtypes: float64(3), int64(7), object(2)
memory usage: 2.0+ GB
